# Full Dataset Showing via Data Wrangler

### Choi Chan Woo (인하대학교 에너지자원환경연구실, Py_Renewable Team)

This Python Code made in 12/15 is for viewing full Dataset used for this project (Since CSV Limitations on Libreoffice and Excel)

In [1]:
import pandas as pd

## Loading Dataset

In [2]:
df_Renewable_Generation = pd.read_csv('../Dataset/Base/한국전력거래소_지역별 시간별 태양광 및 풍력 발전량_2024.csv', encoding='cp949')

In [3]:
df_climate_ASOS_2024_Full = pd.read_csv('../Dataset/Base/OBS_ASOS_TIM_20251215090557.csv', encoding='cp949', low_memory=False)

## Full Dataset View by Data Wrangler

In [4]:
df_Renewable_Generation.head()

,거래일자,거래시간,지역,연료원,전력거래량(MWh)
0,2024-01-01,1,강원도,태양광,0.0
1,2024-01-01,2,강원도,태양광,0.0
2,2024-01-01,3,강원도,태양광,0.0
3,2024-01-01,4,강원도,태양광,0.0
4,2024-01-01,5,강원도,태양광,0.0


In [5]:
df_climate_ASOS_2024_Full.head()

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),지면온도 QC플래그,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,90,속초,2024-01-01 00:00,1.5,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,1396.0,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN
1,90,속초,2024-01-01 01:00,1.2,NaN,NaN,9.0,1.2,NaN,290.0,...,6.0,939.0,NaN,NaN,0.1,NaN,NaN,NaN,NaN,NaN
2,90,속초,2024-01-01 02:00,0.9,NaN,NaN,9.0,0.8,NaN,270.0,...,8.0,789.0,NaN,NaN,0.1,NaN,NaN,NaN,NaN,NaN
3,90,속초,2024-01-01 03:00,1.1,NaN,0.4,NaN,1.3,NaN,290.0,...,8.0,1005.0,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN
4,90,속초,2024-01-01 04:00,1.2,NaN,NaN,9.0,1.4,NaN,290.0,...,9.0,1151.0,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN


## Dataset Properties

In [6]:
# Showing Dataset properties
print("df_Renewable_Generation Info:")
df_Renewable_Generation.info()

df_Renewable_Generation Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166896 entries, 0 to 166895
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   거래일자        166896 non-null  object 
 1   거래시간        166896 non-null  int64  
 2   지역          166896 non-null  object 
 3   연료원         166896 non-null  object 
 4   전력거래량(MWh)  166896 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 6.4+ MB


In [7]:
print("df_climate_ASOS_2024_Full Info:")
df_climate_ASOS_2024_Full.info()

df_climate_ASOS_2024_Full Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852048 entries, 0 to 852047
Data columns (total 38 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   지점             852048 non-null  int64  
 1   지점명            852048 non-null  object 
 2   일시             852048 non-null  object 
 3   기온(°C)         851103 non-null  float64
 4   기온 QC플래그       986 non-null     float64
 5   강수량(mm)        80511 non-null   float64
 6   강수량 QC플래그      162179 non-null  float64
 7   풍속(m/s)        849686 non-null  float64
 8   풍속 QC플래그       1340 non-null    float64
 9   풍향(16방위)       849677 non-null  float64
 10  풍향 QC플래그       1352 non-null    float64
 11  습도(%)          850818 non-null  float64
 12  습도 QC플래그       1275 non-null    float64
 13  증기압(hPa)       850635 non-null  float64
 14  이슬점온도(°C)      850632 non-null  float64
 15  현지기압(hPa)      851102 non-null  float64
 16  현지기압 QC플래그     996 non-null     float64
 1

## Showing Feature names

In [8]:
df_climate_ASOS_2024_Full.columns.tolist()

['지점',
 '지점명',
 '일시',
 '기온(°C)',
 '기온 QC플래그',
 '강수량(mm)',
 '강수량 QC플래그',
 '풍속(m/s)',
 '풍속 QC플래그',
 '풍향(16방위)',
 '풍향 QC플래그',
 '습도(%)',
 '습도 QC플래그',
 '증기압(hPa)',
 '이슬점온도(°C)',
 '현지기압(hPa)',
 '현지기압 QC플래그',
 '해면기압(hPa)',
 '해면기압 QC플래그',
 '일조(hr)',
 '일조 QC플래그',
 '일사(MJ/m2)',
 '일사 QC플래그',
 '적설(cm)',
 '3시간신적설(cm)',
 '전운량(10분위)',
 '중하층운량(10분위)',
 '운형(운형약어)',
 '최저운고(100m )',
 '시정(10m)',
 '지면상태(지면상태코드)',
 '현상번호(국내식)',
 '지면온도(°C)',
 '지면온도 QC플래그',
 '5cm 지중온도(°C)',
 '10cm 지중온도(°C)',
 '20cm 지중온도(°C)',
 '30cm 지중온도(°C)']

In [9]:
df_Renewable_Generation.columns.tolist()

['거래일자', '거래시간', '지역', '연료원', '전력거래량(MWh)']

## NaN Value Checking

In [10]:
# NaN 값 확인
print("NaN values in df_Renewable_Generation:")
print(df_Renewable_Generation.isna().sum())

NaN values in df_Renewable_Generation:
거래일자          0
거래시간          0
지역            0
연료원           0
전력거래량(MWh)    0
dtype: int64


In [11]:
print("NaN values in df_climate_ASOS_2024_Full:")
print(df_climate_ASOS_2024_Full.isna().sum())

NaN values in df_climate_ASOS_2024_Full:
지점                    0
지점명                   0
일시                    0
기온(°C)              945
기온 QC플래그         851062
강수량(mm)          771537
강수량 QC플래그        689869
풍속(m/s)            2362
풍속 QC플래그         850708
풍향(16방위)           2371
풍향 QC플래그         850696
습도(%)              1230
습도 QC플래그         850773
증기압(hPa)           1413
이슬점온도(°C)          1416
현지기압(hPa)           946
현지기압 QC플래그       851052
해면기압(hPa)          1179
해면기압 QC플래그       850827
일조(hr)           389615
일조 QC플래그         463449
일사(MJ/m2)        569651
일사 QC플래그         283405
적설(cm)           831958
3시간신적설(cm)       851343
전운량(10분위)           812
중하층운량(10분위)       11942
운형(운형약어)         732341
최저운고(100m )      424445
시정(10m)            8976
지면상태(지면상태코드)     852048
현상번호(국내식)        801751
지면온도(°C)           2955
지면온도 QC플래그       844670
5cm 지중온도(°C)     598715
10cm 지중온도(°C)    598439
20cm 지중온도(°C)    598948
30cm 지중온도(°C)    598253
dtype: int64


In [12]:
# 지점 출력 및 지점-지점명 매핑
unique_stations = df_climate_ASOS_2024_Full['지점'].unique()
print("Unique Stations in df_climate_ASOS_2024_Full:")
for station in unique_stations:
    station_name = df_climate_ASOS_2024_Full[df_climate_ASOS_2024_Full['지점'] == station]['지점명'].iloc[0]
    print(f"Station Code: {station}, Station Name: {station_name}") 

Unique Stations in df_climate_ASOS_2024_Full:
Station Code: 90, Station Name: 속초
Station Code: 93, Station Name: 북춘천
Station Code: 95, Station Name: 철원
Station Code: 98, Station Name: 동두천
Station Code: 99, Station Name: 파주
Station Code: 100, Station Name: 대관령
Station Code: 101, Station Name: 춘천
Station Code: 102, Station Name: 백령도
Station Code: 104, Station Name: 북강릉
Station Code: 105, Station Name: 강릉
Station Code: 106, Station Name: 동해
Station Code: 108, Station Name: 서울
Station Code: 112, Station Name: 인천
Station Code: 114, Station Name: 원주
Station Code: 115, Station Name: 울릉도
Station Code: 119, Station Name: 수원
Station Code: 121, Station Name: 영월
Station Code: 127, Station Name: 충주
Station Code: 129, Station Name: 서산
Station Code: 130, Station Name: 울진
Station Code: 131, Station Name: 청주
Station Code: 133, Station Name: 대전
Station Code: 135, Station Name: 추풍령
Station Code: 136, Station Name: 안동
Station Code: 137, Station Name: 상주
Station Code: 138, Station Name: 포항
Station Code: 14

In [13]:
# 신재생 에너지 발전량 지역 출력
unique_regions = df_Renewable_Generation['지역'].unique()
print("Unique Regions in df_Renewable_Generation:")
for region in unique_regions:
    print(f"Region: {region}")

Unique Regions in df_Renewable_Generation:
Region: 강원도
Region: 경기도
Region: 경상남도
Region: 경상북도
Region: 광주시
Region: 대구시
Region: 대전시
Region: 부산시
Region: 서울시
Region: 세종시
Region: 울산시
Region: 인천시
Region: 전라남도
Region: 전라북도
Region: 제주도
Region: 충청남도
Region: 충청북도
Region: 육지
Region: 제주


In [14]:
# 신재생 에너지 발전량 에서 특정 지역만 추출 (인천시, 제주)
df_Renewable_Generation_Target = df_Renewable_Generation[df_Renewable_Generation['지역'].isin(['인천시', '제주'])]
df_Renewable_Generation_Target.head()

,거래일자,거래시간,지역,연료원,전력거래량(MWh)
264,2024-01-01,1,인천시,태양광,0.0
265,2024-01-01,2,인천시,태양광,0.0
266,2024-01-01,3,인천시,태양광,0.0
267,2024-01-01,4,인천시,태양광,0.0
268,2024-01-01,5,인천시,태양광,0.0


In [15]:
df_Renewable_Generation_Target.to_csv('../Dataset/한국전력거래소_인천시_제주_태양광_풍력_발전량_2024.csv', index=False, encoding='cp949')

In [16]:
# 기후 데이터에서 102, 112, 201, 98, 99, 119, 202, 203, 184, 185, 188, 189 추출

df_climate_ASOS_2024_Selected = df_climate_ASOS_2024_Full[df_climate_ASOS_2024_Full['지점'].isin([102, 112, 201, 98, 99, 119, 202, 203, 184, 185, 188, 189])]
df_climate_ASOS_2024_Selected.head()

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),지면온도 QC플래그,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
26352,98,동두천,2024-01-01 00:00,-1.5,NaN,NaN,9.0,0.0,NaN,0.0,...,NaN,203.0,NaN,NaN,-0.2,NaN,NaN,NaN,NaN,NaN
26353,98,동두천,2024-01-01 01:00,-1.9,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,275.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN,NaN
26354,98,동두천,2024-01-01 02:00,-2.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,276.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN,NaN
26355,98,동두천,2024-01-01 03:00,-2.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,120.0,NaN,NaN,-0.4,NaN,NaN,NaN,NaN,NaN
26356,98,동두천,2024-01-01 04:00,-2.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,184.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN,NaN


In [17]:
df_climate_ASOS_2024_Selected.to_csv('../Dataset/기후데이터_인천시_제주_2024.csv', index=False, encoding='cp949')

In [18]:
# delete off QC flags in df_climate_ASOS_2024_Selected
columns_to_drop = [col for col in df_climate_ASOS_2024_Selected.columns if col.endswith('QC플래그')]
df_climate_ASOS_2024_Cleaned = df_climate_ASOS_2024_Selected.drop(columns=columns_to_drop)
df_climate_ASOS_2024_Cleaned

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
26352,98,동두천,2024-01-01 00:00,-1.5,NaN,0.0,0.0,96.0,5.3,-2.0,...,NaN,NaN,203.0,NaN,NaN,-0.2,NaN,NaN,NaN,NaN
26353,98,동두천,2024-01-01 01:00,-1.9,NaN,0.0,0.0,96.0,5.1,-2.4,...,NaN,NaN,275.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN
26354,98,동두천,2024-01-01 02:00,-2.1,NaN,NaN,NaN,96.0,5.0,-2.6,...,NaN,NaN,276.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN
26355,98,동두천,2024-01-01 03:00,-2.7,NaN,NaN,NaN,96.0,4.8,-3.2,...,NaN,NaN,120.0,NaN,NaN,-0.4,NaN,NaN,NaN,NaN
26356,98,동두천,2024-01-01 04:00,-2.3,NaN,NaN,NaN,96.0,5.0,-2.8,...,NaN,NaN,184.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439195,203,이천,2024-12-31 19:00,-0.1,NaN,1.3,200.0,51.0,3.1,-9.0,...,NaN,NaN,4811.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
439196,203,이천,2024-12-31 20:00,-1.6,NaN,0.3,0.0,56.0,3.0,-9.2,...,NaN,NaN,4820.0,NaN,NaN,-0.2,NaN,NaN,NaN,NaN
439197,203,이천,2024-12-31 21:00,-2.6,NaN,0.3,0.0,60.0,3.0,-9.3,...,NaN,NaN,4546.0,NaN,NaN,-0.4,NaN,NaN,NaN,NaN
439198,203,이천,2024-12-31 22:00,-2.2,NaN,1.1,250.0,59.0,3.1,-9.1,...,NaN,NaN,3224.0,NaN,NaN,-0.7,NaN,NaN,NaN,NaN


In [19]:
# delete off 지점 in df_climate_ASOS_2024_Selected
columns_to_drop = ['지점']
df_climate_ASOS_2024_Cleaned = df_climate_ASOS_2024_Cleaned.drop(columns=columns_to_drop)
df_climate_ASOS_2024_Cleaned

,지점명,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
26352,동두천,2024-01-01 00:00,-1.5,NaN,0.0,0.0,96.0,5.3,-2.0,1012.8,...,NaN,NaN,203.0,NaN,NaN,-0.2,NaN,NaN,NaN,NaN
26353,동두천,2024-01-01 01:00,-1.9,NaN,0.0,0.0,96.0,5.1,-2.4,1013.1,...,NaN,NaN,275.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN
26354,동두천,2024-01-01 02:00,-2.1,NaN,NaN,NaN,96.0,5.0,-2.6,1013.4,...,NaN,NaN,276.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN
26355,동두천,2024-01-01 03:00,-2.7,NaN,NaN,NaN,96.0,4.8,-3.2,1013.8,...,NaN,NaN,120.0,NaN,NaN,-0.4,NaN,NaN,NaN,NaN
26356,동두천,2024-01-01 04:00,-2.3,NaN,NaN,NaN,96.0,5.0,-2.8,1014.0,...,NaN,NaN,184.0,NaN,NaN,-0.3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439195,이천,2024-12-31 19:00,-0.1,NaN,1.3,200.0,51.0,3.1,-9.0,1013.5,...,NaN,NaN,4811.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
439196,이천,2024-12-31 20:00,-1.6,NaN,0.3,0.0,56.0,3.0,-9.2,1013.9,...,NaN,NaN,4820.0,NaN,NaN,-0.2,NaN,NaN,NaN,NaN
439197,이천,2024-12-31 21:00,-2.6,NaN,0.3,0.0,60.0,3.0,-9.3,1014.2,...,NaN,NaN,4546.0,NaN,NaN,-0.4,NaN,NaN,NaN,NaN
439198,이천,2024-12-31 22:00,-2.2,NaN,1.1,250.0,59.0,3.1,-9.1,1014.3,...,NaN,NaN,3224.0,NaN,NaN,-0.7,NaN,NaN,NaN,NaN
